从这一小节开始，正式进入卷积神经网络的部分

卷积这个操作其实就是为了让全连接层在处理图像数据的时候，能够减少参数的量，不然参数量直接爆炸
* 卷积层实际上就是一种特殊的全连接层

卷积这个操作子其实从数学上理解一开始很难理解，有点抽象。具体的可以对照MLP到卷积的那个公式来理解

卷积的本质就是，利用平移不变形和局部性，从图像中一部分一部分的抽取出信息，这样起到减少参数量的作用

对卷积的理解比较困难的话，可以看b站相关视频

接下来是卷积层的基本实现

In [5]:
import torch
from torch import nn
from d2l import torch as d2l


# 这里实现的其实是一个互相关运算，并不是卷积，卷积要翻一下
# k就是核矩阵
def corr2d(X, K):  #@save
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [6]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

其实这里结合代码能够更好的理解前面的公式
* 前面公式中，卷积核的维度是ab，实际上也就是2delta，这里卷积核的大小是22，所以和delta为1是对的上的
* 由于像素点离散的关系，所以实际上我们在原图像上找小区域和卷积核相乘的时候，我们用的是开区间
* 因为我们这里是遍历，所以实际上并不会取到ab等于负值的情况，因为我们是从最边上开始遍历的

In [ ]:
# 这里就是定义了一个卷积层，和前面讲过的自定义层的操作是完全一样的
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        # 初始化一个卷积核
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        # 用卷积核做前向互相关运算
        return corr2d(x, self.weight) + self.bias

来一个简单的利用，用卷积层检测图片中不同颜色的边缘

In [7]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
K = torch.tensor([[1.0, -1.0]])

这里可以发现，我们中间有一个颜色过度，1 0 1的过程，我们卷积出来的结果1就表示1 0，-1就表示0 1，我们成功的检测到了水平的颜色变化

In [10]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

但是当我们把图片转置之后，竖直的颜色变化就没有被检测到了

In [9]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

接下来看一下，卷积核是咋被学习出来的

In [13]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
# 这里就是直接用了nn定义好的，和我们上面手写的是一样的东西
# 不过Conv2d这个类你要先告诉它，输入和输出的channel是多少
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 实际上就是一张黑白图片
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # print(l)
    # 手写的损失函数，所以sum一下
    # 迭代卷积核
    # 这玩意就是个简单的sgd
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 21.827
epoch 4, loss 7.700
epoch 6, loss 2.946
epoch 8, loss 1.172
epoch 10, loss 0.474


In [14]:
conv2d.weight.data.reshape((1, 2))
# 和我们上面人为定义的卷积核长得差不多

tensor([[ 0.9174, -1.0590]])

* 个人感觉，对于CNN来说，最难的点其实是理解卷积这个操作
* 结合相应的数学公式与实际的卷积操作来理解